In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import re

In [3]:
def broad_filter(res_cases):
    # filter out cases with irrelevant type codes
    types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP']
    filter_cases_type = res_cases[res_cases['A_TYPE'].isin(types)]

    # filter out cases with out of date status
    status = res_cases['A_STATUS'].unique()
    status = status[~np.isin(status, ['WITH', 'VOID','DEN','DISAP','EXP'])]
    filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

    # keep entries with keywords
    keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' housing', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
    pattern = '|'.join(keywords)
    filtered_in = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]

    # remove entries with certain words
    keywords_avoid = ['expand','storage']
    pattern_avoid = '|'.join(keywords_avoid)
    filtered_words = filtered_in[~filtered_in['A_DESCRIPT'].str.contains(pattern_avoid, case=False, na=False)]

    # filter out entries that were last updated over 5 years ago
    filtered_words = filtered_words.copy()
    filtered_words['A_STATUS_D'] = pd.to_datetime(filtered_words['A_STATUS_D'])
    filtered_final = filtered_words[filtered_words['A_STATUS_D'].dt.year>=2020]

    return filtered_final

In [4]:
def normalize_for_regex(term):
    # makes it so string returns a match whether a term has spaces, dashes, both, or neither
    return re.sub(r'[-\s]+', r'\\s*-?\\s*', term)

In [5]:
def extract_units(description):
    # remove square footage references
    description = re.sub(
        r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq\.?\s*ft\.?|sqft)',
        '', description, flags=re.IGNORECASE
    )

    # map variations to standardized types
    term_map = {
        "home": "home", "homes": "home", "house": "home", "houses": "home",
        "duplex": "duplex", "duplexes": "duplex",
        "condo": "condo", "condominium": "condo", "condominiums": "condo", "condos": "condo", 
        "apartment": "apartment", "apartments": "apartment",
        "townhome": "townhouse", "townhomes": "townhouse",
        "townhouse": "townhouse", "townhouses": "townhouse",
        "town home": "townhouse", "town homes": "townhouse",
        "town house": "townhouse", "town houses": "townhouse",
        "multifamily": "multifamily", "multi-family": "multifamily", 
        "multi - family": "multifamily", "multi family": "multifamily",
        "mutifamily": "multifamily", "MF": "multifamily",
        "single family": "single family", "single-family": "single family", 
        "single - family": "single family", "s-f": "single family", "s - f": "single family", "s f": "single family"
    }

    modifiers = ["attached", "detached"]
    suffixes = ["units", "lots", "homes", "houses"]

    housing_pattern = "|".join([normalize_for_regex(term) for term in term_map])
    modifier_pattern = "|".join(modifiers)
    suffix_pattern = "|".join(suffixes)

    # extended match pattern to support both "qty before type" and "type before qty"
    match_pattern = rf'''
    (?:
        # Qty before type
        (?P<qty>\(?\d{{1,4}}\)?)
        (?:\s*[-+&/]?\s*)?
        (?:({modifier_pattern})\s*){{0,2}}?
        (?:\w+\s*){{0,4}}?
        (?P<type>{housing_pattern})
        (?:\s+({modifier_pattern}))?
        (?:\s+(?P<suffix>{suffix_pattern}))?

    |
        # Type before qty
        (?P<type2>{housing_pattern})
        (?:\s+({modifier_pattern}))?
        (?:\s*[-+&/]?\s*)?
        (?:\w+\s*){{0,4}}?
        (?P<qty2>\(?\d{{1,4}}\)?)
        (?:\s+(?P<suffix2>{suffix_pattern}))?

    |
        # Type with quantity in parentheses
        (?P<type3>{housing_pattern})
        (?:\s+\w+){{0,4}}?
        \(\s*(?P<qty3>\d{{1,4}})\s+(?P<suffix3>{suffix_pattern})\s*\)
    )
'''

    matches = re.finditer(match_pattern, description, flags=re.IGNORECASE | re.VERBOSE)
    
    result = []
    for match in matches:
        qty = match.group("qty") or match.group("qty2")
        raw_type = match.group("type") or match.group("type2")
        raw_mod = match.group(2)  # first modifier (position varies)
        raw_suffix = match.group("suffix") or match.group("suffix2")

        if not qty or not raw_type:
            continue  # skip malformed matches

        # normalize type
        norm_key = re.sub(r'[-\s]+', ' ', raw_type.lower()).strip()
        normalized_type = term_map.get(norm_key, norm_key)

        result.append((
            int(qty.strip("()")),
            raw_mod.lower() if raw_mod else None,
            normalized_type,
            raw_suffix.lower() if raw_suffix else None
        ))

    return result

In [6]:
def fill_types(match_results):
    housing_types = ['sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo']
    housing_type_dict = {
        'townhouse': 'sf_attached',
        'home': 'sf_detached', 'single family': 'sf_detached',
        'duplex': 'duplex/triplex',
        'apartment': 'multifamily', 'multifamily': 'multifamily',
        'condo': 'condo'
    }

    row_data = {h_type: 0 for h_type in housing_types}
    for group in match_results:
        quantity = group[0]
        mod = group[1]
        housing = group[2]

        if housing == 'single family' and mod == 'attached':
            row_data['sf_attached'] += quantity
        elif housing in housing_type_dict:
            row_data[housing_type_dict[housing]] += quantity

    return pd.Series(row_data)

In [7]:
durham_dev_filename = input('Please input the name of the Durham developments shapefile: ').strip()
res_cases_raw = gpd.read_file(f'../data/{durham_dev_filename}')
res_filtered = broad_filter(res_cases_raw)
res_filtered['match_results'] = res_filtered['A_DESCRIPT'].apply(extract_units)
housing_counts = res_filtered['match_results'].apply(fill_types)
filtered_final = pd.concat([res_filtered, housing_counts], axis=1)
filtered_final = filtered_final.to_crs('EPSG:4326')

Please input the name of the Durham developments shapefile:  durham_developments


In [51]:
# read data from Data+_2025/data/enrollment_projections/sgr_table_region_2324_20240710.xlsx in Google Drive
'''
read in SGR data -- file paths: 
the current one is from 2024 July 10th, the file is already in data and is named sgr_tables_htype_reg.xlsx
'''
sgr_filename = input('Please enter the file name which includes the table of SGRs by housing type and region: ')
sgr_data = gpd.read_file(f'../data/{sgr_filename}')

Please enter the file name which includes the table of SGRs by housing type and region:  sgr_tables_htype_reg.csv


In [59]:
sgr_data = sgr_data[sgr_data['region'] != '']

In [61]:
sgr_data

,housing_type,region,parcel_ct,du_est_final,dps_2324_all,dps_2223_all,dps_2122_all,dps_2021_all,dps_avg_all,dps_2324_k12,...,sgr_stu_avg_all,sgr_stu_2324_all,sgr_stu_avg_k12,sgr_stu_2324_k12,sgr_dps_avg_k12_es,sgr_dps_avg_k12_ms,sgr_dps_avg_k12_hs,sgr_stu_avg_k12_es,sgr_stu_avg_k12_ms,sgr_stu_avg_k12_hs
4,age_restrict,Central,6,162,0,1,3,3,1.75,0,...,0.00308642,0,0.00308642,0,0.005401235,0.002160494,0.003240741,0.00154321,0.000617284,0.000925926
5,condo,Central,51,812,17,20,20,24,20.25,15,...,0.029556651,0.029556651,0.027709359,0.027093597,0.011853448,0.004741379,0.007112069,0.01385468,0.005541872,0.008312807
6,du_tri,Central,1289,2731,546,590,628,677,610.25,529,...,0.266569018,0.259245694,0.260893434,0.253020883,0.109117538,0.043647014,0.065470524,0.130446717,0.052178688,0.078268029
7,hud,Central,36,306,75,53,72,88,72,72,...,0.290849686,0.310457528,0.284313738,0.300653607,0.114787579,0.04591503,0.068872549,0.142156869,0.056862749,0.08529412
8,manuf,Central,5,12,5,8,4,5,5.5,5,...,0.625,0.5,0.625,0.5,0.229166672,0.091666669,0.137500003,0.3125,0.125,0.1875
9,mf_apt,Central,316,7671,1226,1285,1307,1339,1289.25,1194,...,0.206166074,0.201538265,0.201864168,0.1973667,0.082323037,0.032929216,0.049393822,0.100932084,0.040372834,0.06055925
10,non-res,Central,3507,0,45,43,37,44,42.25,44,...,,,,,,,,,,
11,sf_attach,Central,197,197,6,5,7,10,7,6,...,0.035532996,0.035532996,0.035532996,0.035532996,0.016497461,0.006598984,0.009898476,0.017766498,0.007106599,0.010659899
12,sf_detach,Central,8974,8977,2386,2387,2390,2502,2416.25,2305,...,0.338587493,0.338977396,0.329787225,0.329954326,0.131098926,0.05243957,0.078659356,0.164893612,0.065957442,0.09893617
13,temp,Central,102,774,82,46,29,45,50.5,81,...,0.084625326,0.107235141,0.083333336,0.105943151,0.030846253,0.012338501,0.018507753,0.041666668,0.016666668,0.025


In [63]:
# remove null values and shorten to only use relevant columns
sgr_data = sgr_data.dropna()
sgr_data.rename(columns={'sgr_dps_2324_all.1': 'sgr_dps_avg_k12'}, inplace=True) # because there might be a typo in the file?
sgr_data = sgr_data[['housing_type','region','sgr_dps_avg_k12']]
sgr_data['sgr_dps_avg_k12'] = sgr_data['sgr_dps_avg_k12'].round(4)
sgr_data.set_index(['region', 'housing_type'], inplace=True)

In [65]:
filtered_final.columns

Index(['A_NUMBER', 'A_TYPE', 'A_DATE', 'A_STATUS', 'A_STATUS_D', 'A_PROJECT_',
       'A_DESCRIPT', 'A_USER_ID', 'A_CASE_PLA', 'StatCode', 'AppStatus',
       'AppCode', 'AppType', 'CasePlanne', 'EMAIL', 'ORIG_FID', 'CreationDa',
       'Creator', 'EditDate', 'Editor', 'geometry', 'match_results',
       'sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo',
       'region'],
      dtype='object')

In [67]:
'''
read in shapefile to get geometries for Durham County regions from Data+_2025/QGIS/DPS shapefiles from layers in Google Drive
'''
regions = gpd.read_file(r'../data/durham_regions.geojson')[['region', 'geometry']]
regions = regions.to_crs('EPSG:4326')
#read in geojson with residential developments
# ''' 
# read in geojson with residential developments -- file paths: 
# Leah: /Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/resdev_cases.geojson'''
# res_dev = gpd.read_file(r'/Users/kevan/OneDrive/Desktop/Data+/DPS-Planning/GIS_files/resdev_cases.geojson')
# res_dev = res_dev.to_crs('EPSG:4326')
# res_dev

In [69]:
filtered_final = filtered_final.copy()

for i, geometry in enumerate(regions['geometry']):
    in_geometry = geometry.contains(filtered_final['geometry'])

    region = regions.loc[i, 'region']  # or 'region', depending on your column name
    
    filtered_final.loc[in_geometry, 'region'] = region


In [100]:
hs_full_geo

NameError: name 'hs_full_geo' is not defined

In [71]:
# function to count number of students in each row
def count_students(row): 
        
    htype_map = {
        'sf_detached': 'sf_detach',
        'sf_attached': 'sf_attach',
        'duplex/triplex': 'du_tri',
        'multifamily': 'mf_apt',
        'condo': 'condo'
    }

    region = row['region']

    total = 0
    for col_name, sgr_col in htype_map.items():
        count = row.get(col_name, 0)

        try:
            multiplier = sgr_data.loc[(region, sgr_col), 'sgr_dps_avg_k12']
        except KeyError:
            multiplier = 0

        total += count * multiplier

    return total

In [73]:
filtered_final

,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,EditDate,Editor,geometry,match_results,sf_detached,sf_attached,duplex/triplex,multifamily,condo,region
90,D2000291,PL_MINSP,2020-12-02,APP,2021-09-03,Umstead Grove Conservation Subdivision,"50 Single - family lots, 1 stormwater pond, ad...",JESSICADO,COURTNEYMC,APP,...,2025-07-17,gisproc_sys,POINT (-78.94561 36.07501),"[(50, None, single family, lots)]",50,0,0,0,0,North
139,D2200173,PL_MINSP,2022-05-25,APP,2023-02-01,Pineview Glen Town homes - Mass Grading,Mass grading only site plan for residential to...,JUSTINH,KEAGANSA,APP,...,2025-07-17,gisproc_sys,POINT (-78.81051 35.92776),[],0,0,0,0,0,East
224,D1800378,PL_MINSP,2018-11-28,APP,2020-04-14,Elan Innovation District,Mixe of uses including retail and residential....,JOHNRA,TREYFI,APP,...,2025-07-17,gisproc_sys,POINT (-78.90421 35.99991),[],0,0,0,0,0,Central
242,D2100156,PL_MINSP,2021-06-04,APP,2022-02-02,ALTA Rutherford,Multifamily Apartments,KIMRO,TREYFI,APP,...,2025-07-17,gisproc_sys,POINT (-78.92991 36.01005),[],0,0,0,0,0,Central
265,D1900171,PL_MINSP,2019-05-30,APP,2020-04-03,Ellis Road Phase 3,"37 Townhome units, 102 Attached S-F units (cal...",ROBINSH,COLERE,APP,...,2025-07-17,gisproc_sys,POINT (-78.86167 35.9519),"[(37, None, townhouse, units), (102, None, sin...",248,37,0,0,0,East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21416,D2500149,PL_MINSP,2025-06-23,UN_RE,2025-06-23,Mt. Moriah II-By Right,"156 Units, totaling approximately 65,594 sq. ft.",JONATHANBR,TREYFI,UN_RE,...,2025-07-17,gisproc_sys,POINT (-78.99127 35.94527),[],0,0,0,0,0,Southwest
21423,D2500150,PL_MINSP,2025-06-23,UN_RE,2025-06-23,Mt. Moriah II-Affordable,"152 Units, totaling approximately 59,136 sq. ft.",JONATHANBR,COLERE,UN_RE,...,2025-07-17,gisproc_sys,POINT (-78.99196 35.94594),[],0,0,0,0,0,Southwest
21445,D2500158,PL_MINSP,2025-06-30,UN_RE,2025-06-30,5802 S Miami Blvd Multi-Family,369-unit multi-family building with structured...,SKNOX,TREYFI,UN_RE,...,2025-07-17,gisproc_sys,POINT (-78.85182 35.87137),"[(369, None, multifamily, None)]",0,0,0,369,0,Southeast
21446,D2500159,PL_MINSP,2025-06-30,UN_RE,2025-06-30,Mineral Springs Road Towns,46 townhome lots with 3 open space lots with a...,FRANCISCOM,COLERE,UN_RE,...,2025-07-17,gisproc_sys,POINT (-78.83615 35.96064),"[(46, None, townhouse, lots)]",0,46,0,0,0,East


In [82]:
cols_to_convert = ['sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo']
filtered_final[cols_to_convert] = filtered_final[cols_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0)

In [86]:
print(filtered_final.columns)

Index(['A_NUMBER', 'A_TYPE', 'A_DATE', 'A_STATUS', 'A_STATUS_D', 'A_PROJECT_',
       'A_DESCRIPT', 'A_USER_ID', 'A_CASE_PLA', 'StatCode', 'AppStatus',
       'AppCode', 'AppType', 'CasePlanne', 'EMAIL', 'ORIG_FID', 'CreationDa',
       'Creator', 'EditDate', 'Editor', 'geometry', 'match_results',
       'sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo',
       'region'],
      dtype='object')


In [92]:
sgr_data['sgr_dps_avg_k12'] = pd.to_numeric(sgr_data['sgr_dps_avg_k12'], errors='coerce')

In [94]:
print(sgr_data.dtypes)


sgr_dps_avg_k12    float64
dtype: object


In [96]:
filtered_final['student_gen'] = filtered_final.apply(count_students, axis=1)

In [ ]:
filtered_final[['region', 'sf_detached', 'sf_attached', 'multifamily', 'student_gen']]

In [ ]:
#filtered_final.to_file('resdev_with_stu_proj.geojson', driver='GeoJSON')

In [ ]:
filtered_final["student_gen"].plot.hist(bins=20, edgecolor='black')

plt.xlabel("Student_gen")
plt.ylabel("Frequency")
plt.title("Distribution of student_gen values")
plt.grid(True)

plt.show()

In [ ]:
# read in planning unit shapefile 
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape.geojson')[['OBJECTID', 'geometry']]
pu = pu.to_crs('EPSG:4326')

In [ ]:
# fill missing values
filtered_final['pu'] = filtered_final['pu'].fillna(0).astype(int)

# get counts for each planning unit
pu_gen = filtered_final.groupby('pu')['student_gen'].sum()

In [ ]:
pu['student_gen'] = pu['OBJECTID'].map(pu_gen).fillna(0).round().astype(int)

In [ ]:
# let's fix hs_full_geo
pu = pu.set_index('OBJECTID')

hs_full_geo = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/hs_full_geo.geojson')
hs_full_geo = hs_full_geo.set_index('pu_2324_84')

for i, row in pu.iterrows():
    hs_full_geo.loc[i, 'student_gen'] = row['student_gen']

# hs_full_geo.to_file('hs_full_geo.geojson', driver='GeoJSON')

In [ ]:
pu.to_file('pu_gen.geojson', driver='GeoJSON')